# Imports

In [189]:
import os
import pandas as pd
import librosa
import numpy as np
import scipy.io.wavfile, scipy.signal
from scipy.spatial import distance

# Locations

In [46]:
audio_loc='./resources/dataset/Audio/'
annotation_loc='./resources/dataset/Annotations/processed/'
lookup_loc='./resources/dataset/lookup.csv'
output_loc = './resources/dataset/Annotations/final/'

In [11]:
lut=pd.read_csv(lookup_loc)
"""
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(lut)
"""

"\nwith pd.option_context('display.max_rows', None, 'display.max_columns', None):\n    display(lut)\n"

In [17]:
for file_name in os.listdir(audio_loc):
    if file_name.endswith(".wav"):
        name=file_name[:-4]
        print(audio_loc+file_name)
        annotation=pd.read_csv(annotation_loc+name+'.csv')
        x,sr=librosa.load(audio_loc+file_name, sr=None)
        print(f"The sample rate is {sr}")
        mfcc=librosa.feature.mfcc(x,sr)
        print(mfcc.shape)
        display(annotation)
        break

./resources/dataset/Audio/BNyYiTdqzAY.wav
The sample rate is 48000
(20, 64956)


,sample_start,sample_end,scream_type_start,video_id
0,8149223,8756902,layered,BNyYiTdqzAY
1,8807227,9672062,layered,BNyYiTdqzAY
2,9713256,9799779,clean,BNyYiTdqzAY
3,9807757,10222636,midfry,BNyYiTdqzAY
4,10778974,11365493,layered,BNyYiTdqzAY
5,11440761,11896252,layered,BNyYiTdqzAY
6,11904016,12100608,clean,BNyYiTdqzAY
7,17685339,18759966,midfry,BNyYiTdqzAY
8,18766562,18923160,highfry,BNyYiTdqzAY
9,18930251,19100947,layered,BNyYiTdqzAY


In [2]:
import pandas as pd
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

In [91]:
df=pd.read_csv('./resources/dataset/Annotations/final/annotation.csv')
df

,file_name,sample_start,sample_end,timestamp_start,timestamp_end,scream_type,video_id
0,4600fGWcn9o.wav,1579017,1999183,35.81,45.33,midfry,4600fGWcn9o
1,4600fGWcn9o.wav,2020012,2152867,45.81,48.82,midfry,4600fGWcn9o
2,4600fGWcn9o.wav,2161887,2486176,49.02,56.38,midfry,4600fGWcn9o
3,4600fGWcn9o.wav,2488192,2795520,56.42,63.39,clean,4600fGWcn9o
4,4600fGWcn9o.wav,2797600,3030794,63.44,68.73,midfry,4600fGWcn9o
...,...,...,...,...,...,...,...
1570,0m5fIHHfJTw.wav,6819456,6915072,154.64,156.80,midfry,0m5fIHHfJTw
1571,0m5fIHHfJTw.wav,7056672,7172928,160.02,162.65,midfry,0m5fIHHfJTw
1572,0m5fIHHfJTw.wav,7264704,7286880,164.73,165.24,midfry,0m5fIHHfJTw
1573,0m5fIHHfJTw.wav,7789376,8205120,176.63,186.06,midfry,0m5fIHHfJTw


In [ ]:
df.to_pickle()

# Subsetting the audio files for each annotated section
Takes about 75 minutes

In [107]:
audio_files=[]
video_id=[]
scream_type = []
df['audio'] = ''
for index,row in df.iterrows():
    print(f"row number {index + 1} out of {df.shape[0]}")
    file_name = row['file_name']
    sr,x = scipy.io.wavfile.read('./resources/dataset/Audio/processed/'+file_name)
    #x,sr = librosa.load('./resources/dataset/Audio/processed/'+file_name)

    start = int(row['timestamp_start'] * sr)
    end = int(row['timestamp_end'] * sr)

    
    x_subset = x[start:end]
    x_subset_mono = []
    for x in x_subset:
        x_subset_mono.append(np.mean(x)) # Convert to mono by averaging the L and R files
    x_subset_mono_normalized = x_subset_mono/max(x_subset_mono) #Normalizng audio file
    #df.at[index,'audio'] = x_subset_mono_normalized
    audio_files.append(x_subset_mono_normalized)
    video_id.append(row['video_id'])
    scream_type.append(row['scream_type'])
    """if index == 5:
        break"""
audio_files = np.array(audio_files)
video_id=np.array(video_id)
scream_type=np.array(scream_type)

#print('Creating audio_files.csv')
#np.savetxt('./resources/dataset/Annotations/final/audio_files.csv', audio_files, delimiter=',')
#print('Creating video_id.csv')
#np.savetxt('./resources/dataset/Annotations/final/video_id.csv', video_id, delimiter=',')
#print('Creating scream_type.csv')
#np.savetxt('./resources/dataset/Annotations/final/scream_type.csv', scream_type, delimiter=',')

print('Creating concatenated file')
out=np.stack((audio_files,video_id,scream_type),axis=1)
#np.savetxt('./resources/dataset/Annotations/final/out.csv', out, delimiter=',')
#np.savetxt('./resources/dataset/Annotations/final/out_tab.csv', out, delimiter='\t')
np.save('./resources/dataset/Annotations/final/out.npy', out)

#df.to_csv(output_loc + 'annotation_with_audio_file.csv',header=True, index=False,encoding='utf-8-sig',sep='\t')


row number 1 out of 1575
row number 2 out of 1575
row number 3 out of 1575
row number 4 out of 1575
row number 5 out of 1575
row number 6 out of 1575
row number 7 out of 1575
row number 8 out of 1575
row number 9 out of 1575
row number 10 out of 1575
row number 11 out of 1575
row number 12 out of 1575
row number 13 out of 1575
row number 14 out of 1575
row number 15 out of 1575
row number 16 out of 1575
row number 17 out of 1575
row number 18 out of 1575
row number 19 out of 1575
row number 20 out of 1575
row number 21 out of 1575
row number 22 out of 1575
row number 23 out of 1575
row number 24 out of 1575
row number 25 out of 1575
row number 26 out of 1575
row number 27 out of 1575
row number 28 out of 1575
row number 29 out of 1575
row number 30 out of 1575
row number 31 out of 1575
row number 32 out of 1575
row number 33 out of 1575
row number 34 out of 1575
row number 35 out of 1575
row number 36 out of 1575
row number 37 out of 1575
row number 38 out of 1575
row number 39 out of 

<ipython-input-107-2abfd26561b8>:26: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  audio_files = np.array(audio_files)


Creating concatenated file


In [108]:
d=np.load('./resources/dataset/Annotations/final/out.npy',allow_pickle=True)

In [114]:
d

array([[array([ 0.31913205,  0.3460799 ,  0.35163427, ..., -0.49027985,
       -0.46531571, -0.44335765]),
        '4600fGWcn9o', 'midfry'],
       [array([-0.32005566, -0.29227193, -0.20947124, ..., -0.12982048,
       -0.12099758, -0.13497355]),
        '4600fGWcn9o', 'midfry'],
       [array([-0.43057039, -0.41286966, -0.35982849, ...,  0.32882168,
        0.43792535,  0.60017701]),
        '4600fGWcn9o', 'midfry'],
       ...,
       [array([-0.31916257, -0.41782891, -0.47126682, ...,  0.33689383,
        0.2145146 ,  0.11767937]),
        '0m5fIHHfJTw', 'midfry'],
       [array([0.2268746 , 0.21448408, 0.21395001, ..., 0.02032533, 0.02066103,
       0.02059999]),
        '0m5fIHHfJTw', 'midfry'],
       [array([0.02169866, 0.02298044, 0.02337718, ..., 0.18567461, 0.17600024,
       0.16212958]),
        '0m5fIHHfJTw', 'midfry']], dtype=object)

In [110]:
d.shape

(1575, 3)

In [112]:
d1=pd.DataFrame(d)


array([ 0.31913205,  0.3460799 ,  0.35163427, ..., -0.49027985,
       -0.46531571, -0.44335765])

In [173]:
librosa.feature.mfcc(d[0][0]).shape

(20, 820)

In [176]:
np.array(mean_mfccs(d[0][0])).shape

(20,)

In [71]:
def mean_mfccs(x):
    return [np.mean(feature) for feature in librosa.feature.mfcc(x)]


In [69]:
df

,file_name,sample_start,sample_end,timestamp_start,timestamp_end,scream_type,video_id,audio,mfccs
0,4600fGWcn9o.wav,1579017,1999183,35.81,45.33,midfry,4600fGWcn9o,[ 0.31913205 0.3460799 0.35163427 ... -0.49...,
1,4600fGWcn9o.wav,2020012,2152867,45.81,48.82,midfry,4600fGWcn9o,[-0.32005566 -0.29227193 -0.20947124 ... -0.12...,
2,4600fGWcn9o.wav,2161887,2486176,49.02,56.38,midfry,4600fGWcn9o,[-0.43057039 -0.41286966 -0.35982849 ... 0.32...,
3,4600fGWcn9o.wav,2488192,2795520,56.42,63.39,clean,4600fGWcn9o,[0.14267403 0.29718925 0.41744743 ... 0.702734...,
4,4600fGWcn9o.wav,2797600,3030794,63.44,68.73,midfry,4600fGWcn9o,[-0.05304117 -0.11307108 -0.08723716 ... 0.06...,
...,...,...,...,...,...,...,...,...,...
1570,0m5fIHHfJTw.wav,6819456,6915072,154.64,156.80,midfry,0m5fIHHfJTw,[ 0.02751244 0.0298471 0.00125126 ... -0.34...,
1571,0m5fIHHfJTw.wav,7056672,7172928,160.02,162.65,midfry,0m5fIHHfJTw,[-0.24770348 -0.30777917 -0.33234657 ... -0.23...,
1572,0m5fIHHfJTw.wav,7264704,7286880,164.73,165.24,midfry,0m5fIHHfJTw,[-0.31916257 -0.41782891 -0.47126682 ... 0.33...,
1573,0m5fIHHfJTw.wav,7789376,8205120,176.63,186.06,midfry,0m5fIHHfJTw,[0.2268746 0.21448408 0.21395001 ... 0.020325...,


In [130]:
d=np.load('./resources/dataset/Annotations/final/out.npy',allow_pickle=True)
df1=pd.DataFrame(d,columns=['audio','video_id','scream_type'])

df1['mfccs']=''
for i in range(len(df1)):
    df1['mfccs'][i] = mean_mfccs(df1['audio'][i])


df1['zcr']=''
df1['average_zcr']=''
for i in range(len(df)):
    zcr=librosa.feature.zero_crossing_rate(df1['audio'][i])
    df1['zcr'][i] = zcr
    df1['average_zcr'][i] = np.mean(zcr)



In [131]:
df1

,audio,video_id,scream_type,mfccs,zcr,average_zcr
0,"[0.3191320535905026, 0.34607989745780815, 0.35...",4600fGWcn9o,midfry,"[-22.951915178084942, 147.9998225518942, -42.6...","[[0.02490234375, 0.0380859375, 0.04638671875, ...",0.064327
1,"[-0.32005565919447077, -0.29227193492155723, -...",4600fGWcn9o,midfry,"[-28.35545909304096, 144.549112698403, -38.467...","[[0.0205078125, 0.0400390625, 0.046875, 0.0522...",0.055764
2,"[-0.43057039094210636, -0.412869655446028, -0....",4600fGWcn9o,midfry,"[-20.199514020813762, 146.58846360897235, -43....","[[0.0498046875, 0.0546875, 0.05810546875, 0.04...",0.064544
3,"[0.14267403180028687, 0.29718924527726065, 0.4...",4600fGWcn9o,clean,"[-14.670658634010156, 168.33928471014002, -63....","[[0.03173828125, 0.0439453125, 0.064453125, 0....",0.071517
4,"[-0.05304116946928312, -0.1130710776085696, -0...",4600fGWcn9o,midfry,"[-9.732367427497497, 172.67747628214553, -58.6...","[[0.03857421875, 0.06005859375, 0.08740234375,...",0.06845
...,...,...,...,...,...,...
1570,"[0.027512436292611468, 0.029847102267525256, 0...",0m5fIHHfJTw,midfry,"[-34.14331133370637, 167.9931998843823, -45.44...","[[0.0224609375, 0.03369140625, 0.0458984375, 0...",0.055238
1571,"[-0.24770348216193122, -0.3077791680654317, -0...",0m5fIHHfJTw,midfry,"[-32.03566299599044, 162.47606410873925, -45.3...","[[0.02734375, 0.05078125, 0.07470703125, 0.068...",0.054126
1572,"[-0.3191625720999786, -0.41782891323587756, -0...",0m5fIHHfJTw,midfry,"[-2.041026111374771, 158.19804578029135, -43.3...","[[0.0283203125, 0.0419921875, 0.060546875, 0.0...",0.069536
1573,"[0.22687459944456312, 0.21448408459730828, 0.2...",0m5fIHHfJTw,midfry,"[-38.34468935177947, 165.83730795548183, -43.0...","[[0.0185546875, 0.029296875, 0.03759765625, 0....",0.06322


In [132]:
lut = pd.read_csv('./resources/dataset/lookup_new.csv')
lut

,band_name,album_name,song_name,release_year,duration_seconds,video_id,bit_depth,bitrate,channels,sample_rate
0,A Job for a Cowboy,Ruination,Entombment of a Machine,2013,247.520000,nOJHbpijw3o,16,1536000,2,48000
1,Abbath,Abbath,Nebular Ravens Winter,2016,256.696599,QJyTW1-Piwo,16,1411200,2,44100
2,Abbath,Abbath,Ashes Of The Damned,2016,238.097415,K5pMoSECagE,16,1411200,2,44100
3,Abbath,Outstrider,Harvest Pyre,2019,274.088345,T_qa-wNbC4w,16,1411200,2,44100
4,Abbath,Abbath,Winter Bane,2016,409.878640,yLdJSQDmVXw,16,1411200,2,44100
...,...,...,...,...,...,...,...,...,...,...
115,Whitechapel,Our Endless War,The Saw Is the Law,2014,267.656417,rQqgjVAklHQ,16,1411200,2,44100
116,Wintersun,Time I,Sons of Winter and Stars,2012,811.119456,ttoJ-Ro0L04,16,1411200,2,44100
117,Wintersun,Wintersun 2.0,Beautiful Death 2.0,2017,513.207438,TNg-mfVnCWk,16,1411200,2,44100
118,Wintersun,Wintersun 2.0,Starchild 2.0,2017,491.566440,oyuOjCfCsm4,16,1411200,2,44100


In [134]:
df2=df1.merge(lut[['band_name','video_id','song_name']], 'inner','video_id')
df2

,audio,video_id,scream_type,mfccs,zcr,average_zcr,band_name,song_name
0,"[0.3191320535905026, 0.34607989745780815, 0.35...",4600fGWcn9o,midfry,"[-22.951915178084942, 147.9998225518942, -42.6...","[[0.02490234375, 0.0380859375, 0.04638671875, ...",0.064327,Textures,Storm Warning
1,"[-0.32005565919447077, -0.29227193492155723, -...",4600fGWcn9o,midfry,"[-28.35545909304096, 144.549112698403, -38.467...","[[0.0205078125, 0.0400390625, 0.046875, 0.0522...",0.055764,Textures,Storm Warning
2,"[-0.43057039094210636, -0.412869655446028, -0....",4600fGWcn9o,midfry,"[-20.199514020813762, 146.58846360897235, -43....","[[0.0498046875, 0.0546875, 0.05810546875, 0.04...",0.064544,Textures,Storm Warning
3,"[0.14267403180028687, 0.29718924527726065, 0.4...",4600fGWcn9o,clean,"[-14.670658634010156, 168.33928471014002, -63....","[[0.03173828125, 0.0439453125, 0.064453125, 0....",0.071517,Textures,Storm Warning
4,"[-0.05304116946928312, -0.1130710776085696, -0...",4600fGWcn9o,midfry,"[-9.732367427497497, 172.67747628214553, -58.6...","[[0.03857421875, 0.06005859375, 0.08740234375,...",0.06845,Textures,Storm Warning
...,...,...,...,...,...,...,...,...
1570,"[0.027512436292611468, 0.029847102267525256, 0...",0m5fIHHfJTw,midfry,"[-34.14331133370637, 167.9931998843823, -45.44...","[[0.0224609375, 0.03369140625, 0.0458984375, 0...",0.055238,Lamb of God,Now You've Got Something to Die For
1571,"[-0.24770348216193122, -0.3077791680654317, -0...",0m5fIHHfJTw,midfry,"[-32.03566299599044, 162.47606410873925, -45.3...","[[0.02734375, 0.05078125, 0.07470703125, 0.068...",0.054126,Lamb of God,Now You've Got Something to Die For
1572,"[-0.3191625720999786, -0.41782891323587756, -0...",0m5fIHHfJTw,midfry,"[-2.041026111374771, 158.19804578029135, -43.3...","[[0.0283203125, 0.0419921875, 0.060546875, 0.0...",0.069536,Lamb of God,Now You've Got Something to Die For
1573,"[0.22687459944456312, 0.21448408459730828, 0.2...",0m5fIHHfJTw,midfry,"[-38.34468935177947, 165.83730795548183, -43.0...","[[0.0185546875, 0.029296875, 0.03759765625, 0....",0.06322,Lamb of God,Now You've Got Something to Die For


# Splitting the 20 MFCCs into indididual columns

In [179]:
df2[['mfcc1','mfcc2','mfcc3','mfcc4','mfcc5','mfcc6','mfcc7','mfcc8','mfcc9','mfcc10','mfcc11','mfcc12','mfcc13','mfcc14','mfcc15','mfcc16','mfcc17','mfcc18','mfcc19','mfcc20']] = pd.DataFrame(df2.mfccs.tolist(), index= df2.index)
#df3 = pd.DataFrame(df2['mfccs'].to_list(), columns=['mfcc1','mfcc2','mfcc3','mfcc4','mfcc5','mfcc6','mfcc7','mfcc8','mfcc9','mfcc10','mfcc11','mfcc12','mfcc13','mfcc14','mfcc15','mfcc16','mfcc17','mfcc18','mfcc19','mfcc20'])

In [201]:
df2['scream_type'].unique()

array(['midfry', 'clean', 'highfry', 'lowfry', 'layered'], dtype=object)

## Mapping the scream types into 0-4

In [202]:
df2['scream_type'] = df2['scream_type'].map({'midfry' :0, 'clean' :1, 'highfry' :2,'lowfry' :3, 'layered' :4}).astype(int)

# Creating train-test splits based on band name

In [221]:
from sklearn.model_selection import GroupShuffleSplit
train_inds, test_inds = next(GroupShuffleSplit(test_size=.33, n_splits=2, random_state = 0).split(df2, groups=df2['band_name']))

train = df2.iloc[train_inds]
test = df2.iloc[test_inds]

## Breakout by scream type for each split

In [223]:
train.groupby('scream_type')[['audio']].count()

,audio
scream_type,
0,772
1,75
2,105
3,40
4,65


In [224]:
test.groupby('scream_type')[['audio']].count()

,audio
scream_type,
0,339
1,90
2,65
3,13
4,11


## Bands present in each split

In [225]:
train['band_name'].unique()

array(['Veil of Maya', 'Gojira', 'Children of Bodom', 'Cannibal Corpse',
       'Behemoth', 'Parkway Drive', "Be'lakor", 'Tesseract', 'Abbath',
       'Opeth', 'Godless', 'Lamb of God', 'Dark Tranquillity',
       'Suicide Silence', 'Ne Obliviscaris', 'Born of Osiris',
       'After The Burial', 'Children Of Bodom', 'Of Mice & Men',
       'Enslaved', 'Ensiferum', 'Suffocation'], dtype=object)

In [226]:
test['band_name'].unique()

array(['Textures', 'Wintersun', 'Rings of Saturn', 'Decapitated',
       'Thy Art Is Murder', 'Death', 'In Flames', 'Immortal',
       'Amon Amarth', 'Nevermore', 'Slayer', 'Slipknot'], dtype=object)

# Converting dataframe to numpy arrays for training

In [227]:
x_train = train[['mfcc1','mfcc2','mfcc3','mfcc4','mfcc5','mfcc6','mfcc7','mfcc8','mfcc9','mfcc10','mfcc11','mfcc12','mfcc13','mfcc14','mfcc15','mfcc16','mfcc17','mfcc18','mfcc19','mfcc20','average_zcr']].to_numpy()
y_train = train[['scream_type']].to_numpy()
x_test = test[['mfcc1','mfcc2','mfcc3','mfcc4','mfcc5','mfcc6','mfcc7','mfcc8','mfcc9','mfcc10','mfcc11','mfcc12','mfcc13','mfcc14','mfcc15','mfcc16','mfcc17','mfcc18','mfcc19','mfcc20','average_zcr']].to_numpy()
y_test = test[['scream_type']].to_numpy()

In [228]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(1057, 21)
(1057, 1)
(518, 21)
(518, 1)


# KNN Classifier code

In [229]:
#Importing the required modules
import numpy as np
from scipy.stats import mode
 
#Euclidean Distance
def eucledian(p1,p2):
    dist = np.sqrt(np.sum((p1-p2)**2))
    return dist
 
#Function to calculate KNN
def predict(x_train, y , x_input, k):
    # x_train, y_train, x_input are all arrays containing arrays with input values
    # k is number of nearest neighbours to be included in the voting
    #1) There is no structured method to find the best value for “K”. We need to find out with various values by trial and error and assuming that training data is unknown.
    #2) Choosing smaller values for K can be noisy and will have a higher influence on the result.
    #3) Larger values of K will have smoother decision boundaries which mean lower variance but increased bias. Also, computationally expensive.
    #4) Another way to choose K is though cross-validation. One way to select the cross-validation dataset from the training dataset. Take the small portion from the training dataset and call it a validation dataset, and then use the same to evaluate different possible values of K. This way we are going to predict the label for every instance in the validation set using with K equals to 1, K equals to 2, K equals to 3.. and then we look at what value of K gives us the best performance on the validation set and then we can take that value and use that as the final setting of our algorithm so we are minimizing the validation error .
    #5) In general, practice, choosing the value of k is k = sqrt(N) where N stands for the number of samples in your training dataset.#
    op_labels = []
     
    #Loop through the Datapoints to be classified
    for item in x_input: 
         
        #Array to store distances
        point_dist = []
         
        #Loop through each training Data
        for j in range(len(x_train)): 
            distances = eucledian(np.array(x_train[j,:]) , item)#distance.euclidean()# 
            #Calculating the distance
            point_dist.append(distances) 
        point_dist = np.array(point_dist) 
         
        #Sorting the array while preserving the index
        #Keeping the first K datapoints
        dist = np.argsort(point_dist)[:k] 
         
        #Labels of the K datapoints from above
        labels = y[dist]
         
        #Majority voting
        lab = mode(labels) 
        lab = lab.mode[0]
        op_labels.append(lab)
 
    return op_labels

# Run KNN

In [230]:
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
from numpy.random import randint
import matplotlib.pyplot as plt
import numpy as np
k = int(np.round(np.sqrt(x_train.shape[0])))

y_pred = predict(x_train,y_train,x_test,k)

#Checking the accuracy
score = accuracy_score(y_test, y_pred)
score

0.637065637065637

In [231]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred,labels=[0,1,2,3,4]) #'midfry' :0, 'clean' :1, 'highfry' :2,'lowfry' :3, 'layered' :4}
from sklearn.metrics import accuracy_score 
print ("Accuracy : ", accuracy_score(y_test, y_pred))
cm

Accuracy :  0.637065637065637


array([[327,   3,   8,   0,   1],
       [ 89,   1,   0,   0,   0],
       [ 61,   2,   2,   0,   0],
       [ 13,   0,   0,   0,   0],
       [ 10,   0,   1,   0,   0]])

In [2]:
!pip install spleeter

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  Using cached spleeter-2.3.0-py3-none-any.whl (51 kB)
  Using cached librosa-0.8.0-py3-none-any.whl
  Using cached llvmlite-0.36.0-cp39-cp39-macosx_10_9_x86_64.whl (18.5 MB)
  Using cached norbert-0.2.1-py2.py3-none-any.whl (11 kB)
  Using cached tensorflow-2.5.0-cp39-cp39-macosx_10_11_x86_64.whl (195.7 MB)
  Using cached tensorflow_estimator-2.5.0-py2.py3-none-any.whl (462 kB)
  Using cached typing_extensions-3.7.4.3-py3-none-any.whl (22 kB)
  Using cached grpcio-1.34.1-cp39-cp39-macosx_10_10_x86_64.whl (3.7 MB)
  Using cached keras_nightly-2.5.0.dev2021032900-py2.py3-none-any.whl (1.2 MB)
  Using cached h2-4.1.0-py3-none-any.whl (57 kB)
  Using cached hpack-4.0.0-py3-none-any.whl (32 kB)
  Using cached hyperframe-6.0.1-py3-none

In [8]:
from spleeter.separator import Separator

# Using embedded configuration.
separator = Separator('spleeter:2stems')

# # Using custom configuration file.
# separator = Separator('/path/to/config.json')

# Use audio loader explicitly for loading audio waveform :
from spleeter.audio.adapter import AudioAdapter

audio_loader = AudioAdapter.default()
sample_rate = 44100
waveform, _ = audio_loader.load('./resources/working_data/-xYZM04JxnQ_original.wav', sample_rate=sample_rate)

# Perform the separation :
prediction = separator.separate(waveform)

INFO:tensorflow:Apply unet for vocals_spectrogram
INFO:tensorflow:Apply unet for accompaniment_spectrogram
INFO:tensorflow:Restoring parameters from pretrained_models/2stems/model


In [9]:
prediction['vocals']

array([[0., 0.],
       [0., 0.],
       [0., 0.],
       ...,
       [0., 0.],
       [0., 0.],
       [0., 0.]], dtype=float32)

In [10]:
import soundfile as sf
sf.write('./resources/working_data/'+'test'+'_vocal.wav', prediction['vocals'], 44100)